# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [85]:
%pip install xlsxwriter

Note: you may need to restart the kernel to use updated packages.


In [86]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [87]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [88]:
from secrets import IEX_CLOUD_API_TOKEN

In [ ]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol}?token={IEX_CLOUD_API_TOKEN}'
print(api_url)
data = requests.get(api_url).json()
data

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [90]:
data[0]['year1ChangePercent']

0.2330574514785948

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [91]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

for symbol_string in symbol_strings:
    print(symbol_string)
    
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [92]:
# df = pd.DataFrame(columns = my_columns)
df = pd.DataFrame()
new_df1 = pd.DataFrame()
new_df2 = pd.DataFrame()

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote,advanced_stats/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    # print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    half = len(data)//2
    data1, data2 = data[:half], data[half:]

    for data_arr in data1:
        new_df = pd.DataFrame(
            [[
                data_arr['symbol'],
                data_arr['latestPrice']
            ]],columns=['Ticker', 'Price']
        )
        new_df1 = pd.concat([new_df1, new_df], ignore_index=True)

    for data_arr in data2:
        new_df = pd.DataFrame(
            [[
                data_arr['year1ChangePercent']
            ]],columns=['One-Year Price Return']
        )
        new_df2 = pd.concat([new_df2, new_df], ignore_index=True)

df = new_df1.join(new_df2)
df

# 10.2s

/tmp/ipykernel_20491/3191536411.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df2 = pd.concat([new_df2, new_df], ignore_index=True)


,Ticker,Price,One-Year Price Return
0,A,132.03,-0.071303
1,AAL,15.13,-0.046629
2,AAP,61.10,-0.558867
3,AAPL,182.52,0.233057
4,ABBV,178.09,0.218474
...,...,...,...
500,YUM,138.32,0.095662
501,ZBH,129.18,0.056933
502,ZBRA,274.99,-0.086412
503,ZION,39.03,-0.167472


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [93]:
df.sort_values('One-Year Price Return', ascending=False, inplace=True)
df = df[:50]
df.reset_index(inplace=True, drop=True)
df

,Ticker,Price,One-Year Price Return
0,NVDA,788.17,2.332150
1,LB,79.92,2.281287
2,LLY,769.54,1.369266
3,AVGO,1296.37,1.293706
4,AMD,176.52,1.213417
5,PHM,105.74,0.975891
6,ANET,267.66,0.956007
7,LRCX,928.50,0.952780
8,GE,153.33,0.854264
9,AMZN,174.99,0.826237


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [94]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
        print(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")

portfolio_input()


10000000


In [95]:
new_df3 = pd.DataFrame()

position_size = float(portfolio_size) / len(df.index)

for i in range(0, len(df.index)):
      
    new_df = pd.DataFrame(
            [[
                math.floor(position_size/df.loc[i, 'Price'])
            ]],columns=['Number of Shares to Buy']
        )
    new_df3 = pd.concat([new_df3, new_df], ignore_index=True)

df = df.join(new_df3)
df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,NVDA,788.17,2.332150,253
1,LB,79.92,2.281287,2502
2,LLY,769.54,1.369266,259
3,AVGO,1296.37,1.293706,154
4,AMD,176.52,1.213417,1133
5,PHM,105.74,0.975891,1891
6,ANET,267.66,0.956007,747
7,LRCX,928.50,0.952780,215
8,GE,153.33,0.854264,1304
9,AMZN,174.99,0.826237,1142


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [96]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_df = pd.DataFrame()
new_df1 = pd.DataFrame()
new_df2 = pd.DataFrame()

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote,advanced_stats/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    # print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    half = len(data)//2
    data1, data2 = data[:half], data[half:]

    for data_arr in data1:
        new_df = pd.DataFrame(
            [[
                data_arr['symbol'],
                data_arr['latestPrice']
            ]],columns=['Ticker', 'Price']
        )
        new_df1 = pd.concat([new_df1, new_df], ignore_index=True)

    for data_arr in data2:
        new_df = pd.DataFrame(
            [[
                data_arr['year1ChangePercent'],
                data_arr['month6ChangePercent'],
                data_arr['month3ChangePercent'],
                data_arr['month1ChangePercent']
            ]],
            columns = [
                'One-Year Price Return',
                'Six-Month Price Return',
                'Three-Month Price Return',
                'One-Month Price Return'
                ]
        )
        new_df2 = pd.concat([new_df2, new_df], ignore_index=True)

df = new_df1.join(new_df2)
df

/tmp/ipykernel_20491/2109332914.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df2 = pd.concat([new_df2, new_df], ignore_index=True)


,Ticker,Price,One-Year Price Return,Six-Month Price Return,Three-Month Price Return,One-Month Price Return
0,A,132.03,-0.071303,0.112398,0.066645,0.002963
1,AAL,15.13,-0.046629,0.023681,0.229082,0.080714
2,AAP,61.10,-0.558867,-0.112151,0.166600,-0.075363
3,AAPL,182.52,0.233057,0.012966,-0.043510,-0.062475
4,ABBV,178.09,0.218474,0.234642,0.298415,0.063224
...,...,...,...,...,...,...
500,YUM,138.32,0.095662,0.084421,0.083215,0.055122
501,ZBH,129.18,0.056933,0.129462,0.137114,0.066898
502,ZBRA,274.99,-0.086412,0.018142,0.217524,0.073802
503,ZION,39.03,-0.167472,0.174838,0.146208,-0.077835


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [97]:
# time_periods = [
#     'One-Year',
#     'Six-Month',
#     'Three-Month',
#     'One-Month'
#     ]

new_df3 = pd.DataFrame()

for i in range(0, len(df.index)):
    # new_df_row = pd.DataFrame()

    # for time_period in time_periods:
    #     new_df = pd.DataFrame(
    #             [[
    #                 stats.percentileofscore(df[f'{time_period} Price Return'],df.loc[i,f'{time_period} Price Return'], nan_policy='omit')/100
    #             ]],
    #             columns=[
    #                 f'{time_period} Return Percentile'
    #                 ]
    #         )
        
    #     new_df_row = new_df_row.join(new_df)  

    new_df = pd.DataFrame(
            [[
                stats.percentileofscore(df['One-Year Price Return'],df.loc[i,'One-Year Price Return'], nan_policy='omit')/100,
                stats.percentileofscore(df['Six-Month Price Return'],df.loc[i,'Six-Month Price Return'], nan_policy='omit')/100,
                stats.percentileofscore(df['Three-Month Price Return'],df.loc[i,'Three-Month Price Return'], nan_policy='omit')/100,
                stats.percentileofscore(df['One-Month Price Return'],df.loc[i,'One-Month Price Return'], nan_policy='omit')/100
            ]],
            columns=[
                'One-Year Return Percentile',
                'Six-Month Return Percentile',
                'Three-Month Return Percentile',
                'One-Month Return Percentile'
                ]
        )
    
    new_df3 = pd.concat([new_df3, new_df], ignore_index=True)

df = df.join(new_df3)
df

,Ticker,Price,One-Year Price Return,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,One-Year Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile
0,A,132.03,-0.071303,0.112398,0.066645,0.002963,0.227926,0.537988,0.383984,0.369610
1,AAL,15.13,-0.046629,0.023681,0.229082,0.080714,0.258727,0.312115,0.868583,0.786448
2,AAP,61.10,-0.558867,-0.112151,0.166600,-0.075363,0.006160,0.094456,0.698152,0.069815
3,AAPL,182.52,0.233057,0.012966,-0.043510,-0.062475,0.671458,0.285421,0.090349,0.096509
4,ABBV,178.09,0.218474,0.234642,0.298415,0.063224,0.659138,0.772074,0.942505,0.696099
...,...,...,...,...,...,...,...,...,...,...
500,YUM,138.32,0.095662,0.084421,0.083215,0.055122,0.507187,0.474333,0.431211,0.659138
501,ZBH,129.18,0.056933,0.129462,0.137114,0.066898,0.451745,0.583162,0.611910,0.720739
502,ZBRA,274.99,-0.086412,0.018142,0.217524,0.073802,0.203285,0.293634,0.852156,0.753593
503,ZION,39.03,-0.167472,0.174838,0.146208,-0.077835,0.119097,0.677618,0.642710,0.063655


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [98]:
from statistics import mean

time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
    ]

new_df4 = pd.DataFrame()

for row in df.index:
    momentum_percentiles = []
    
    for time_period in time_periods:
        momentum_percentiles.append(df.loc[row,f'{time_period} Return Percentile'])
    
    new_df = pd.DataFrame(
            [[
                mean(momentum_percentiles)
            ]],
            columns=[
                'HQM Score'
                ]
        )
    
    new_df4 = pd.concat([new_df4, new_df], ignore_index=True)

df = df.join(new_df4)
df

,Ticker,Price,One-Year Price Return,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,One-Year Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile,HQM Score
0,A,132.03,-0.071303,0.112398,0.066645,0.002963,0.227926,0.537988,0.383984,0.369610,0.379877
1,AAL,15.13,-0.046629,0.023681,0.229082,0.080714,0.258727,0.312115,0.868583,0.786448,0.556468
2,AAP,61.10,-0.558867,-0.112151,0.166600,-0.075363,0.006160,0.094456,0.698152,0.069815,0.217146
3,AAPL,182.52,0.233057,0.012966,-0.043510,-0.062475,0.671458,0.285421,0.090349,0.096509,0.285934
4,ABBV,178.09,0.218474,0.234642,0.298415,0.063224,0.659138,0.772074,0.942505,0.696099,0.767454
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,138.32,0.095662,0.084421,0.083215,0.055122,0.507187,0.474333,0.431211,0.659138,0.517967
501,ZBH,129.18,0.056933,0.129462,0.137114,0.066898,0.451745,0.583162,0.611910,0.720739,0.591889
502,ZBRA,274.99,-0.086412,0.018142,0.217524,0.073802,0.203285,0.293634,0.852156,0.753593,0.525667
503,ZION,39.03,-0.167472,0.174838,0.146208,-0.077835,0.119097,0.677618,0.642710,0.063655,0.375770


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [99]:
df.sort_values('HQM Score', ascending=False, inplace=True)
df = df[:50]
df.reset_index(inplace=True, drop=True)
df

,Ticker,Price,One-Year Price Return,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,One-Year Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile,HQM Score
0,NVDA,788.17,2.332150,0.673114,0.618030,0.316403,1.000000,0.991786,1.000000,0.997947,0.997433
1,RL,188.44,0.627689,0.676835,0.544107,0.350437,0.950719,0.993840,0.993840,1.000000,0.984600
2,PVH,135.95,0.693798,0.690082,0.577080,0.141095,0.963039,0.995893,0.995893,0.942505,0.974333
3,LLY,769.54,1.369266,0.395680,0.295415,0.224255,0.995893,0.956879,0.940452,0.989733,0.970739
4,AMAT,197.16,0.771823,0.342432,0.323446,0.184249,0.975359,0.907598,0.958932,0.975359,0.954312
5,URI,658.23,0.493442,0.475301,0.423872,0.141472,0.899384,0.981520,0.987680,0.944559,0.953285
6,GE,153.33,0.854264,0.353104,0.282829,0.179915,0.983573,0.913758,0.938398,0.973306,0.952259
7,LRCX,928.50,0.952780,0.385107,0.301217,0.118149,0.985626,0.944559,0.944559,0.897331,0.943018
8,NFLX,583.56,0.803059,0.364893,0.220837,0.185640,0.979466,0.928131,0.858316,0.977413,0.935832
9,TDG,1199.89,0.666898,0.437144,0.236745,0.119457,0.958932,0.975359,0.882957,0.899384,0.929158


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [100]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
        print(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")

portfolio_input()

10000000


In [101]:
new_df5 = pd.DataFrame()

position_size = float(portfolio_size) / len(df.index)

for i in range(0, len(df.index)):
      
    new_df = pd.DataFrame(
            [[
                math.floor(position_size/df.loc[i, 'Price'])
            ]],columns=['Number of Shares to Buy']
        )
    new_df5 = pd.concat([new_df5, new_df], ignore_index=True)

df = df.join(new_df5)
df

,Ticker,Price,One-Year Price Return,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,One-Year Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile,HQM Score,Number of Shares to Buy
0,NVDA,788.17,2.332150,0.673114,0.618030,0.316403,1.000000,0.991786,1.000000,0.997947,0.997433,253
1,RL,188.44,0.627689,0.676835,0.544107,0.350437,0.950719,0.993840,0.993840,1.000000,0.984600,1061
2,PVH,135.95,0.693798,0.690082,0.577080,0.141095,0.963039,0.995893,0.995893,0.942505,0.974333,1471
3,LLY,769.54,1.369266,0.395680,0.295415,0.224255,0.995893,0.956879,0.940452,0.989733,0.970739,259
4,AMAT,197.16,0.771823,0.342432,0.323446,0.184249,0.975359,0.907598,0.958932,0.975359,0.954312,1014
5,URI,658.23,0.493442,0.475301,0.423872,0.141472,0.899384,0.981520,0.987680,0.944559,0.953285,303
6,GE,153.33,0.854264,0.353104,0.282829,0.179915,0.983573,0.913758,0.938398,0.973306,0.952259,1304
7,LRCX,928.50,0.952780,0.385107,0.301217,0.118149,0.985626,0.944559,0.944559,0.897331,0.943018,215
8,NFLX,583.56,0.803059,0.364893,0.220837,0.185640,0.979466,0.928131,0.858316,0.977413,0.935832,342
9,TDG,1199.89,0.666898,0.437144,0.236745,0.119457,0.958932,0.975359,0.882957,0.899384,0.929158,166


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [102]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [103]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [104]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['One-Year Price Return', percent_template],
    'D': ['Six-Month Price Return', percent_template],
    'E': ['Three-Month Price Return', percent_template],
    'F': ['One-Month Price Return', percent_template],
    'G': ['One-Year Return Percentile', percent_template],
    'H': ['Six-Month Return Percentile', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Return Percentile', percent_template],
    'K': ['HQM Score', percent_template],
    'L': ['Number of Shares to Buy', integer_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [105]:
writer.close()